# Population Estimates

-----

### Requirements

Table called 'data'

#### Observations & Dimensions

The `observations` should be apparent.

The required dimensions are:

* **Geography** - get this from the top row "title", cut off the non place names.
* **Time** - use dates from left hand column
* **CDID** - from across the top

-----
Notes:

* This should be pretty straight forward in data baker, but consider doing it in pure pandas after you're done, we'll need
to do that from time to time.

To do it with pandas, replace the load cell with
```
tidy_sheet = pd.read_excel('./sources/pop.xlsx', sheetname='Sheet1')
```


In [1]:
# from databaker.framework import * # made redundant by gssutils
from gssutils import *
import pandas as pd # for post-processing

# put your input-output files here
inputfile = "./sources/pop.xls"
outputfile = "pop_results.csv"
# tidy_sheets = [] # not needed becuase only one tab. TODO could program it so it checks how many tabs are present before deciding to use the variable

# load tab data
tidy_sheet = loadxlstabs(inputfile) # calling this a tidy_sheet off the bat threw me off a little. it's still a list just with one element in there?

# print(type(tidy_sheet))

Loading file ./sources/pop.xls which has size 12288 bytes
Table names: ['data']
<class 'list'>


In [2]:
# create dictionary/function to remove unwanted text from geography name
# TODO redo this as a funciton that uses the in operator and searches whether a header contains a particular country, so it's less susceptable to subtle changes. 
geography_dict = {
'Scotland population mid-year estimate': 'Scotland'
,'Great Britain population mid-year estimate': 'Great Britain'
,'England population mid-year estimate': 'England'
,'United Kingdom population mid-year estimate': 'United Kingdom'
,'England and Wales population mid-year estimate': 'England and Wales'
,'Northern Ireland population mid-year estimate': 'Northern Ireland'
,'Wales population mid-year estimate': 'Wales'
}
# if item isn't in result then kill program

# define function
# TODO could do with some exception capture
def return_country(x):
    if 'Scotland' in x:
        return('Scotland')
    elif 'England and Wales' in x: # has to go before England or Wales separately
        return('England and Wales')
    elif 'Great Britain' in x: # has to go before England or Wales separately
        return('Great Britain')
    elif 'United Kingdom' in x: # has to go before England or Wales separately
        return('United Kingdom')
    elif 'Northern Ireland' in x: # has to go before England or Wales separately
        return('Northern Ireland')
    elif 'England' in x: # has to go before England or Wales separately
        return('England')
    elif 'Wales' in x: # has to go before England or Wales separately
        return('Wales')
    # TODO put in an else statement to force a fail/stop program if this doens't work...or use dictionary

# 

In [3]:
# might have to do this because currently tidy_sheets is type list.
tab = tidy_sheet[0]

# define a selection of cells as the observations
# bear in mind this will include observations that have been aggregated to region level
observations = tab.excel_ref('B8:H8').expand(DOWN).is_not_blank().is_not_whitespace() # TODO databaler's waffle technique might be better here

# dimensions
geography = tab.excel_ref('B1').expand(RIGHT).is_not_blank().is_not_whitespace() # just do the minimum 
time = tab.excel_ref('A8').expand(DOWN).is_not_blank().is_not_whitespace()
CDID = tab.excel_ref('B2').expand(RIGHT).is_not_blank().is_not_whitespace()

dimensions = [ 
        HDim(geography, "geography", DIRECTLY, ABOVE, cellvalueoverride=geography_dict), # need to remove anything not a country name #TODO find out if i can input a function here instead of dictionary
        HDim(time, "Year", DIRECTLY, LEFT), 
        HDim(CDID, "CDID", DIRECTLY, ABOVE),
    ]

# Now we process these relationship for this tab (this code rarely changes)
cs = ConversionSegment(tab, dimensions, observations) 

# preview
# savepreviewhtml(cs) # this didn't work for some reason not sure why, said look up depth passed


In [4]:
# convert conversion segment into a dataframe. Can do this straight away because there are no other conversation segments to bring 
# print(type(cs))
data = cs.topandas()


<class 'databaker.jupybakeutils.ConversionSegment'>


In [6]:
data[:10]
# TODO the OBS should be converted to int64...pandas assumes float


,OBS,CDID,geography,Year
0,5235600.0,SCPOP,Scotland,1971
1,54387600.0,GBPOP,Great Britain,1971
2,46411700.0,ENPOP,England,1971
3,55928000.0,UKPOP,United Kingdom,1971
4,49152000.0,EWPOP,England and Wales,1971
5,1540400.0,NIPOP,Northern Ireland,1971
6,2740300.0,WAPOP,Wales,1971
7,5230600.0,SCPOP,Scotland,1972
8,54557700.0,GBPOP,Great Britain,1972
9,46571900.0,ENPOP,England,1972
